# Early Detection of Crop Stress From Thermal Infrared Imagery
![Hydrosat](https://uploads-ssl.webflow.com/61e4aee27ac4a95d23ab9609/61e9d6f5d6578e8c7c0cca8f_solutions-thermal-min.jpg)

In [1]:
import base64
import pandas as pd
import geopandas as gpd
import json
import numpy as np
import os
import pystac
import requests
import xarray as xr
import rioxarray as rxr
import rasterio as rio

from botocore.exceptions import ClientError
from datetime import date
from datetime import datetime
from distutils.command import sdist
from matplotlib import pyplot as plt
from pyproj.crs import CRS
from pystac_client import Client
from pprint import pprint
from shapely.geometry import box, mapping, Point, Polygon

# Project specific packages
from FH_Hydrosat import FH_StackedDataset
from FH_Hydrosat import FH_Hydrosat
from herbie import FastHerbie
import synoptic.services as ss
from synoptic.services import stations_timeseries

os.environ['USE_PYGEOS'] = '0'


/var/folders/zs/pstrmbpn6t75t6hdl9_v8_8w0000gn/T/ipykernel_26848/2513628167.py:3: UserWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas will still use PyGEOS by default for now. To force to use and test Shapely 2.0, you have to set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In a future release, GeoPandas will switch to using Shapely by default. If you are using PyGEOS directly (calling PyGEOS functions on geometries from GeoPandas), this will then stop working and you are encouraged to migrate from PyGEOS to Shapely 2.0 (https://shapely.readthedocs.io/en/latest/migration_pygeos.html).
  import geopandas as gpd
/opt/miniconda3/envs/hydrosat/lib/python3.9/site-packages/synoptic/accessors.py:28: UserWarning: map making not available without Brian's cartopy_tools
  warnings.warn("map making not available wit

### <span style="color:green">FUNCTIONS.  No user editing is required.</span>

In [27]:
# ---------------------------------------------------------------------------------

def create_clip_polygon(geom, ds, buffer):
    """
    Create a polygon that we will use for clipping the big dataset.

    Parameters:
    ----------
    geom: dict
        Dictionary of lat and lon of center point of AOI.

    Returns:
    -------
    poly: dataframe
        Contains geometry of a square to be used for clipping.
    """
    # Using the point coords defined earlier, create a df with the point geometry.
    p_geom = Point(geom['coordinates'][0], geom['coordinates'][1])
    point_df = gpd.GeoDataFrame({'geometry':[p_geom]}, crs=CRS.from_epsg(4326))

    # Define a buffer size (for each side of the point.
    # Reproject the point df and create the new polygon.
    raster_crs = CRS.from_wkt(ds.spatial_ref.crs_wkt)
    buffer_dist = buffer # 1km in local UTM zone

    # create a square buffer
    poly_df = point_df.to_crs(raster_crs).buffer(buffer_dist, cap_style = 3) 

    return(poly_df)

# ---------------------------------------------------------------------------------

def stac_tile_search(collection, geom, start_date, end_date):
    """
    Log into STAC and search for a specified image collection.

    Parameters:
    ----------
    collection: list
        List of tiles found in STAC.

    geom: 
        Point location to search.

    start_date, end_date: str
        Dates to search between.

    Returns:
    -------
    """

    print (collection, geom, start_date, end_date)
    search = catalog.search(
        collections = collection,
        intersects = geom,
        datetime = [start_date, end_date],
        max_items = 500
    )

    #found_items = list(search.items()) # for pystac-client >= 0.4.0
    found_items = list(search.get_all_items()) # for pystac-client < 0.4.0

    # Filter out only the newest version of MODIS.
    if collection == 'prepped_inputs_mcd43a4':
        version_str = '061'
        new_list = [i for i in found_items if version_str in i.id]
        found_items = new_list

    found_items.reverse() # make the results ascending in time

    num_tiles = len(found_items)
    print ("Searching {} colllection .... images available: {}\n".format(collection, num_tiles))

    return (found_items, num_tiles)

# ---------------------------------------------------------------------------------

def create_aoi_image_stack(items, num_tiles, geom, asset, buffer):
    '''
    Gets images, stacks them and sorts them by date and clips them down to a smaller
    AOI size.

    Parameters:
    -----------
    itmes: list 
        List of available images.
    num_tiles: int
        Number of tiles to download (days)
    asset: str
        Name of asset to get.
    geom: x,y coords
        Coordinates around which to build a polygon
    buffer: int
        Buffer around the x,y for creating the AOI rectangele.  In meters.

    Returns:
    --------
    aoi_stack_ds: dataset as FH_StackedDataset object.
        Stack of images clipped to AOI.

    '''
    images = FH_Hydrosat(items[:num_tiles], asset=asset)

    # Stacks all the files into a dataset and then return a FH_StackedDataset object.
    stacked_images = images.stack()
    # Sort the dataset by time.
    ds = stacked_images.ds.sortby('time')

    # Create polygon Area of Interest (AOI for which to sample.).
    clip_poly_df = create_clip_polygon(geom, ds, buffer=buffer)
    # Use AOI polygon to clip the dataset dwon to size and make it into a FH_StackedDataset object.
    clipped = FH_StackedDataset(ds.rio.clip(clip_poly_df.geometry))
    aoi_stack_ds = clipped.ds

    return (aoi_stack_ds, clip_poly_df)

 # ---------------------------------------------------------------------------------   
def extract_time_series(items, bbox, tol, var_name, asset, band):
    '''
    Uses FH_Hydrosat class method point_time_series_from_items()
    to extract only a time-series.

    Parameters:
    -----------
    items: list
        Image items returned from STAC search.
    bbox: 
        Bounding box of coordinates for seacrh site.
    tol: int
        A search parameter in meters for finding point data.
    var_name: str
        Dataframe column name for data extracted.
    asset: str
        Search parameter for type of asset to be searched.
    
    Returns:
    -------
    lst_df: dataframe
        Dataframe containing date time series.
    '''
    # Sample the LST items.
    lst_res = FH_Hydrosat(items, asset=asset)

    # Set the point for time-series extraction.
    point_wgs84 = Point(box(*bbox).centroid.x, box(*bbox).centroid.y)
    print ("----", point_wgs84)
    
    # Extract time-series data using function.
    band = int(band) # band needs to be an int because it comes in as a string.
    lst_k  = lst_res.point_time_series_from_items(point_wgs84, tol=tol, nproc=6, pad=0, band=band) 

    # Create a datetime dataframe
    lst_dt = lst_res.datetime
    lst_df = pd.DataFrame({var_name: lst_k,
                       'datetime': pd.to_datetime(lst_dt)}).sort_values(by='datetime')
    
    # Get the date in the correct/consistent format.
    lst_df['date'] = [t.to_pydatetime().strftime('%Y-%m-%d') for t in lst_df['datetime']]
    lst_df['date'] = pd.to_datetime(lst_df['date'])
    lst_df.drop(columns='datetime', inplace=True)
    lst_df.set_index('date', drop=True, inplace=True)
    
    return (lst_df)

# ---------------------------------------------------------------------------------

def get_hrrr_point_data(lat, lon, start, days):
    ''' 
    Fetch met data from the HRRR model system.
    Herbie package needs to be installed:
    https://herbie.readthedocs.io/en/stable/
    ${HOME}/.config/herbie/config.toml

    Parameters:
    ----------
    lat, lon: float
        Coords from which to pull grid cell data. 
    start: str
        This is the start date for which data will be retrieved.
    days: int
        Number of days of data to retrieve.

    Return:
    -------
    ds_point: xarray dataset
        Met data for specified point.
    '''
    start_hr_time = "{} 17:30".format(start)
    # Create a range of dates
    DATES = pd.date_range(
        start=start_hr_time,
        periods=days,
        freq="1D",
    )

    # Define forecast lead time (or analysis).
    fxx = range(0, 1)

    # -----------------------------------------------
    # For long time-series analyses
    # -----------------------------------------------
    FH = FastHerbie(DATES, model="hrrr", fxx=fxx)

    FH.download("TMP:2 m")
    t_ds = FH.xarray("TMP:2 m", remove_grib=False)

    FH.download("DPT:2 m")
    td_ds = FH.xarray("DPT:2 m", remove_grib=False)
    # -----------------------------------------------
    # For real-time fetching of HRRR data.
    # -----------------------------------------------
    # Make FastHerbie Object.
    # FH = FastHerbie(DATES, model="hrrr", fxx=fxx)
    # Read a subset of the data with xarray.
    # ds = FH.xarray("TMP:2 m", remove_grib=False)
    # -----------------------------------------------

    # Get data values nearest single point
    t_ds_point = t_ds.herbie.nearest_points(points=(lon, lat))
    td_ds_point = td_ds.herbie.nearest_points(points=(lon, lat))
   
    return (t_ds_point, td_ds_point)

# ---------------------------------------------------------------------------------
def get_synoptic_obs(station):
    # ---------------------------------------------------------------------
    # Request a single station by listing the station id.  
    # ---------------------------------------------------------------------
    params = dict(
        stid=station,
        vars=['air_temp', 'dew_point_temperature'],
        start=datetime(2020, 4, 1),
        end=datetime(2020, 10, 30)
    )
    data = stations_timeseries(verbose='HIDE', **params)

    # Filter data to get only 00z obs.
    sfc_obs = data.loc[(data.index.hour == 17) & (data.index.minute == 30)]
    sfc_obs.drop(columns='dew_point_temperature_set_1', inplace=True)
    # convert datetime column to just date
    sfc_obs['time'] = sfc_obs.index.date
    sfc_obs.reset_index(inplace=True)
    sfc_obs.set_index('time', inplace=True)

    # Change date format to be consistent.
    sfc_obs.index = pd.to_datetime(sfc_obs.index)
    sfc_obs.index = sfc_obs.index.strftime('%Y-%m-%d')

    return (sfc_obs)

# ---------------------------------------------------------------------------------

def read_ameriflux(data_path):
    df = pd.read_csv(data_path, header=2, na_values=[-9999])

    # Save value column names
    value_cols = df.columns[2:]

    # Convert timestamp objects
    df['start'] = df['TIMESTAMP_START'].apply(
        lambda x: datetime.strptime(str(x), "%Y%m%d%H%M")
        )
    df['end'] = df['TIMESTAMP_END'].apply(
        lambda x: datetime.strptime(str(x), "%Y%m%d%H%M")
        )

    # Drop NA
    df = df.dropna(subset=value_cols, how='all')

    df = df.set_index('start')
    col_order = (['end', 'TIMESTAMP_START', 'TIMESTAMP_END'] 
                 + value_cols.to_list())
    df = df[col_order]

    return df

### <span style="color:green">Login to STAC.</span>

In [3]:
# Open credentials file.
with open('../secrets/creds.json') as f:
    creds = json.loads(f.read())

# Endecode the `username:password` combination 
# and use it to authorize access to the STAC API given by the `cat_url` 
# endpoint.userpass = f"{creds['username']}:{creds['password']}"
userpass = f"{creds['username']}:{creds['password']}"
b64 = base64.b64encode(userpass.encode()).decode()
headers = {'Authorization':'Basic ' + b64}

cat_url = 'https://fusion-stac.hydrosat.com'
catalog = Client.open(cat_url, headers)

### <span style="color:green">Set up analysis.  User edits required in cell below.</span>

In [136]:
# ---------------------------------------------------------------------------
# USER EDITS REQUIRED -------------------------------------------------------
# ---------------------------------------------------------------------------

analysis = 'Corn'

# Insitu point locations
insitu_met_points = {
    'Rice': [38.1235, -121.5490],
    'Corn': [38.1091, -121.5351],
    'Alfalfa': [38.0992, -121.4993]
}

# Center point of crop fields.
crop_center_points = {
    'Rice': [-121.550, 38.123],
    'Corn': [-121.536, 38.109],
    'Alfalfa': [-121.501, 38.099],
    'Ames': [-93.701, 42.000],
}

bbox = [-121.52, 38.11, -121.54, 38.09]

# Define size of AOI.
#    Length    x    Width 
# (buffer * 2) x (buffer * 2)
buffer = 250

# Specify dates & hour for LST analysis.
# start_date = "2016-08-13T00:00:00Z"
# end_date = "2022-05-26T00:00:00Z"
start = "2021-08-01"  
end = "2021-08-30"  
hr_s = "00:00:00"  
hr_e = '23:59:59'
# ---------------------------------------------------------------------------
# ---------------------------------------------------------------------------

# Create dict of coords.  Will be used to create a polygon for our AOI.
geom = {'type': 'Point', 'coordinates': crop_center_points[analysis]} 

# We need the total num of days for the analysis to get hrrr data.
f = start.split("-")
l = end.split("-")

f_dt = date(int(f[0]), int(f[1]), int(f[2]))  
l_dt = date(int(l[0]), int(l[1]), int(l[2]))  
num_days = (l_dt - f_dt)
num_days = num_days.days

# Need to do some formating for various requirements.
hr_s_form = "T{}Z".format(hr_s)
hr_e_form = "T{}Z".format(hr_e)
start_date = start + hr_s_form
end_date = end + hr_e_form


### <span style="color:green">Get Fused LST data. </span>

In [137]:
# Collection to search for.
collections = ["starfm_predictions_modis_landsat", "pydms_sharpened_landsat"]
asset = 'lst'

# Search STAC for available images.
(found_items, num_tiles) = stac_tile_search(collections, geom, start_date, end_date)

# Extract data from AOI. 
(aoi_lst_da, aoi_df) = create_aoi_image_stack(found_items, num_tiles, geom, asset, buffer)

# Extract data from point.
# (lst_time_series_df) = (extract_time_series(found_items, bbox, int(40), 'fused_lst', asset, int(0)))

aoi_lst_da


['starfm_predictions_modis_landsat', 'pydms_sharpened_landsat'] {'type': 'Point', 'coordinates': [-121.536, 38.109]} 2021-08-01T00:00:00Z 2021-08-30T23:59:59Z
Searching ['starfm_predictions_modis_landsat', 'pydms_sharpened_landsat'] colllection .... images available: 30



<xarray.DataArray (time: 30, band: 1, y: 25, x: 25)>
dask.array<getitem, shape=(30, 1, 25, 25), dtype=float32, chunksize=(1, 1, 25, 25), chunktype=numpy.ndarray>
Coordinates:
  * band         (band) int64 1
  * x            (x) float64 6.281e+05 6.281e+05 ... 6.286e+05 6.286e+05
  * y            (y) float64 4.219e+06 4.219e+06 ... 4.219e+06 4.219e+06
  * time         (time) object 2021-08-01T11:59:59.500000+00:00 ... 2021-08-3...
    spatial_ref  int64 0
Attributes:
    AREA_OR_POINT:       Area
    OVR_RESAMPLING_ALG:  NEAREST
    scale_factor:        1.0
    add_offset:          0.0
    _FillValue:          nan

### Get AOI mean values

In [ ]:
# Get mean of all pixels for each time-step.
aoi_mean_lst_da = aoi_lst_da.isel(band=0).mean(dim=["x", "y"])

# Convert to dataframe.
aoi_mean_lst_df = aoi_mean_lst_da.to_dataframe(name='starfm_lst')
# Change date format to be consistent.
aoi_mean_lst_df.index = pd.to_datetime(aoi_mean_lst_df.index)
aoi_mean_lst_df.index = aoi_mean_lst_df.index.strftime('%Y-%m-%d')

# Create df with a complete list of dates.
days = pd.date_range(start, end)
dates_df = pd.DataFrame({'time': days})
dates_df['time'] = pd.to_datetime(dates_df.time)
dates_df.set_index('time', inplace=True)
# Change date format to be consistent.
dates_df.index = dates_df.index.strftime('%Y-%m-%d')

# Merge each df into master dataframe.
# Merge on index (date) and keep all rows from both dfs (inner join).
lst_df = pd.merge(
    dates_df, aoi_mean_lst_df, 
    left_index=True, right_index=True, how = 'outer')  

# Interpolate missing values.
interpd_aoi_lst_df = lst_df.interpolate(method='linear', inplace=False)
# Drop uneeded columns.
interpd_aoi_lst_df.drop(columns=['band', 'spatial_ref'], inplace=True)

# Plot up a time series of of the daily AOI mean LST.
fig, ax = plt.subplots(figsize=(12, 6))

ax = plt.plot(interpd_aoi_lst_df.starfm_lst, marker='o', markersize=4, c='blue')

plt.title('AOI Mean of Fused LST Time Series')
plt.grid(True)
plt.ylabel('Fused LST [K] (20 m)')
plt.xticks(rotation=45)
plt.show()


### Get HRRR 2-m Temperature data and put it in it's own dataframe.

In [ ]:
# Get the hrrr 2mT and Td data.
hrrr, hrrr_td = get_hrrr_point_data(lat, lon, start, num_days)

# Convert to dataframe and clean it up.
# Get a dataset of point hrrr and convert to dataframe for easier use. 
hrrr_df = hrrr.to_dataframe()
hrrr_td_df = hrrr_td.to_dataframe()

# Make date consistent in format.
hrrr_df.reset_index(inplace=True)
hrrr_df['date'] = pd.to_datetime(hrrr_df['time'].dt.date)

hrrr_td_df.reset_index(inplace=True)
hrrr_td_df['date'] = pd.to_datetime(hrrr_td_df['time'].dt.date)

# Set the index to date just like the other dataframes.
hrrr_df.set_index('time', inplace=True)
# Change date format to be consistent.
hrrr_df.index = hrrr_df.index.strftime('%Y-%m-%d')

# Set the index to date just like the other dataframes.
hrrr_td_df.set_index('time', inplace=True)
# Change date format to be consistent.
hrrr_td_df.index = hrrr_td_df.index.strftime('%Y-%m-%d')

## Get Surface met obs from Synoptic Data.

In [ ]:
station = 'KAMW'
obs_df = get_synoptic_obs(station)

# Create df with a complete list of dates.
days = pd.date_range(start, end)
dates_df = pd.DataFrame({'time': days})
dates_df['time'] = pd.to_datetime(dates_df.time)
dates_df.set_index('time', inplace=True)
# Change date format to be consistent.
dates_df.index = dates_df.index.strftime('%Y-%m-%d')

met_obs_df = pd.merge(
    dates_df, obs_df, 
    left_index=True, right_index=True, how = 'outer')  


### Now, we'll calculate CATD using the LST and 2-m Temperature.

In [ ]:
# CATD will be calculated as:
# (CATD = LST - 2 m Temperature)

# Calculate CATD.
catd_df = pd.DataFrame()
catd_df['catd'] = interpd_aoi_lst_df['starfm_lst'].sub(hrrr_df['t2m'])

# Plot up a time series of of the daily AOI mean LST.
fig, ax = plt.subplots(figsize=(12, 6))

ax = plt.plot(catd_df, marker='s', markersize=8, c='green')

plt.title('CATD Time Series for All Pixels')
plt.grid(True)
plt.ylabel('CATD (20 m)')
plt.xticks(rotation=45)
plt.show()


### Calculate NDVI

In [ ]:
# Surface reflectance is extracted for and NDVI is calculated.
    
# Search STAC for available images from collection..
collection = "starfm_predictions_modis_s2"
(found_items, num_tiles) = stac_tile_search(collection, geom, start_date, end_date)

# Extract red band data from aoi.. 
asset='surface_reflectance_red'
(aoi_red_da) = create_aoi_image_stack(found_items, num_tiles, geom, asset, buffer)

# Extract nir band data from aoi. 
asset='surface_reflectance_nir'
(aoi_nir_da) = create_aoi_image_stack(found_items, num_tiles, geom, asset, buffer)

# Get mean of all pixels for each time-step.
aoi_mean_red_da = aoi_red_da.isel(band=0).mean(dim=["x", "y"])
aoi_mean_nir_da = aoi_nir_da.isel(band=0).mean(dim=["x", "y"])

# Calculate ndvi.
ndvi_da = (aoi_mean_nir_da - aoi_mean_red_da)  / (aoi_mean_nir_da + aoi_mean_red_da)

# Create our own dataframe instead of using to_dataframe which takes
# forever dealing with the dask graphs.
# ndvi_vals = ndvi_da.values()
# ndvi_dates = ndvi_da.time().values
# ndvi_df = pd.DataFrame({"time": ndvi_dates, "ndvi": ndvi_vals})


# Convert to dataaframe
ndvi_df = ndvi_da.to_dataframe(name='ndvi')

# Change date format to be consistent.
# Set the index to date just like the other dataframes.

# Change date format to be consistent.
ndvi_df.index = ndvi_df.index.strftime('%Y-%m-%d')


### Plot Mean CATD and Mean NDVI

In [ ]:
# Plot up a time series of of the daily AOI mean LST.
fig, ax1 = plt.subplots(figsize=(12, 6))
# Plot on 2 y-axes.
ax2 = ax1.twinx()  

ax1.plot(catd_df.index, catd_df.catd, marker='s', markersize=8, c='green', label='CATD')
ax2.plot(ndvi_df.index, ndvi_df.ndvi, marker='s', markersize=8, c='blue', label='NDVI')

#ax1.set_ylabel("Fused CATD", color='green', fontsize=14)
#ax2.set_ylabel(label=label, color='blue', fontsize=14)
ax1.set(ylim=(-25, 25), ylabel="CATD")
ax2.set( ylabel="NDVI")

ax1.grid(True) 

plt.title('AOI Mean CATD & Mean NDVI')
plt.xticks(rotation=45)
ax1.set_xticklabels(ax1.get_xticks(), rotation = 45)

ax1.legend(loc=2)
ax2.legend(loc=1)
plt.show()

### CWSI-EB (Crop Water Stress Index) - Katimbo et al (2022) in Agricultural Water Management.

CWSI = ((Tc - Ta)a - (Tcl - Ta)U) / ((Tcu - Ta)L - (Tcl - Ta)L)

    Tc = canopy Temp (fused LST)
    Ta = ambient air temp (HRRR 2mT)
    Tcl = Tc of well transpiring veg (from aoi.max)
    Tcu = Tc of non-transpiring veg (from aoi.min)
    
    (Tcl - Ta)L = A + B * VPD

        VPD = e - es
        e = 6.11 × 10 exp(7.5 × Td / 237.3 + Td )
        es = 6.11 × 10 exp(7.5 × T / 237.3 + T )
        A = intercept of a regression of (Tc - Ta) and VPD
        B = slope of a regression of (Tc - Ta) and VPD

    (Tc - Ta)a = A + B * VPG
    
        VPG = Ta - (Ta + A)
        A = intercept of a regression of (Tc - Ta) and VPD
        B = slope of a regression of (Tc - Ta) and VPD



In [ ]:
import numpy as np
from sklearn.linear_model import LinearRegression

t_canopy_da = aoi_lst_da
t_air = hrrr_df['t2m']
t_d = hrrr_df['td']
t_canopy_mx = aoi_lst_da.max()
t_canopy_mn = aoi_lst_da.min()

e = 6.11e((7.5 * t_d) / (273.3 + t_d))
es = 6.11e((7.5 * t_air) / (273.3 + t_air))
vpd = e - es

model = LinearRegression()
model.fit(t_canopy_da, vpd)
model = LinearRegression().fit(t_canopy_da, vpd)
r_sq = model.score(t_canopy_da, vpd)
a = model.intercept_
b = model.coef_

vpg = t_air - (t_air + a)

cwsi = ((a + b * vpg) - (t_canopy_mx - t_air)) / ((t_canopy_mn - t_air) - (a + b * vpd))


### Calculate VPD and plot CATD vs. VPD.

In [ ]:
# Calculate Vapor Pressure Deficit (VPD).
# e = vapor pressure
# es = saturated vapor pressure.
e = 6.11 * 10 ** ((7.5 * (met_obs_df['dew_point_temperature'])) / (237.3 + (met_obs_df['dew_point_temperature'])))
es = 6.11 * 10 ** ((7.5 * (met_obs_df['air_temp'])) / (237.3 + (met_obs_df['air_temp'])))
vpd = es - e

# Convert LST from K to C.
c_lst_df = lst_df['starfm_lst'] - 273.15

# Calculate CATD this time in celcius.
catd_obs_df = c_lst_df.sub(met_obs_df['air_temp'])

# Plot CATD vs. VPD
plt.scatter(vpd['2020-07-01':'2020-08-30'], catd_obs_df['2020-07-01':'2020-08-30'], c='blue', alpha=0.5)
plt.xlabel('Vaopr Pressure Deficit (VPD)')
plt.ylabel('CATD')
plt.title('AMES - CATD (Mean of 500 sqm AOI) vs. Observed VPD')
plt.show()


## <span style="color:green">Set paths to met data csv files.</span>


In [20]:
# Define paths to data
data_path = os.path.join('../data', 'Ameriflux')
bi1_data_path = os.path.join(data_path, 
                             'AMF_US-Bi1_BASE-BADM_8-5',
                             'AMF_US-Bi1_BASE_HH_8-5.csv')
bi2_data_path = os.path.join(data_path,
                             'AMF_US-Bi2_BASE-BADM_13-5',
                             'AMF_US-Bi2_BASE_HH_13-5.csv')
ds3_data_path = os.path.join(data_path,
                             'AMF_US-DS3_BASE-BADM_1-5',
                             'AMF_US-DS3_BASE_HH_1-5.csv')

## <span style="color:green">Get Temp and Vapor Pressure from insitsu site.</span>
### I want to take the single temp value for each time step and broadcast it across
### a 25 x 25 array to match the LST dataarray.  Then, once the two dataarrays are of
### the same size and shape, I can subract the two to get a catd for each grid cell for each time step.

In [224]:
# Get observed insitsu 'TA' and 'VPD_PI' from local csv files.

# AMF_US-Bi1_BASE_HH_8-5.csv
bi1_df = read_ameriflux(bi1_data_path)
# AMF_US-Bi2_BASE_HH_13-5
# bi2_df = read_ameriflux(bi2_data_path)
# AMF_US-DS3_BASE-BADM_1-5
# bi3_df = read_ameriflux(ds3_data_path)

bi1_df.index = pd.to_datetime(bi1_df.index)

#bi1_df.TA
#bi1_df.VPD_PI

da_list = []
img_stack = []

# Loop through each time step of the LST da.
for i in (aoi_lst_da.time):

    # Convert date to proper format
    dt = pd.to_datetime(i.item())
    # Get temp from the csv.
    ta = bi1_df.TA.iloc[bi1_df.index.get_loc(dt, method='nearest')]

    # Build dataarray of temp.

    # Create one datarray per time step.
    time = pd.date_range(dt, periods=1)

    # Use coords from LST da.
    temp_da = (xr.DataArray(
        data=ta,
        coords={'time': time, 'y': aoi_lst_da.y, 'x': aoi_lst_da.x, 'band':1},
        dims=['time', 'y', 'x'],
        name='TA',
        ))
    
    # Stick each da into a list.
    da_list.append(temp_da)

# Concat the lists of das along time.
ta_stack = xr.concat(da_list, dim="time")

/var/folders/zs/pstrmbpn6t75t6hdl9_v8_8w0000gn/T/ipykernel_26848/1682375658.py:24: FutureWarning: Passing method to DatetimeIndex.get_loc is deprecated and will raise in a future version. Use index.get_indexer([item], method=...) instead.
  ta = bi1_df.TA.iloc[bi1_df.index.get_loc(dt, method='nearest')]


## <span style="color:green">Subtract the dataarrays</span>

In [238]:
lst = aoi_lst_da.isel(band=0)

print (ta_stack.coords)

# I'm expecting the subtraction to be done across time.
result_da = lst - ta_stack

# Result is that is somehow reduces time to 0.  
# The two dataarrays are the same shape ((time: 0, y: 25, x: 25))
result_da


Coordinates:
  * time     (time) datetime64[ns] 2021-08-01T11:59:59.500000 ... 2021-08-30T...
  * y        (y) float64 4.219e+06 4.219e+06 4.219e+06 ... 4.219e+06 4.219e+06
  * x        (x) float64 6.281e+05 6.281e+05 6.282e+05 ... 6.286e+05 6.286e+05
    band     int64 1


<xarray.DataArray (time: 0, y: 25, x: 25)>
dask.array<sub, shape=(0, 25, 25), dtype=float64, chunksize=(0, 25, 25), chunktype=numpy.ndarray>
Coordinates:
  * x            (x) float64 6.281e+05 6.281e+05 ... 6.286e+05 6.286e+05
  * y            (y) float64 4.219e+06 4.219e+06 ... 4.219e+06 4.219e+06
  * time         (time) object 
    band         int64 1
    spatial_ref  int64 0

In [ ]:

new_da = xr.DataArray()
for i in list:
    da = xr.full_like(aoi_lst_da, fill_value=i, dtype=None)
    print (da)

## Calculate CATD On each cell in the stacked AOI.

In [ ]:
# Rename dimension to match LST's da.
xa = met_da.rename(new_name_or_name_dict={"start":"time"})
# Add dimensions equal to the LST dataarray so the two da's align.
met_x_da = xa.expand_dims(dim={"band":1, "y": 25, "x":25}, axis=[1,2,3])

#a, b = xr.align(met_x_da, aoi_lst_da, join='exact')




In [ ]:
# catd = LST - Isitsu Air Temp.
#aoi_catd_da = aoi_lst_da - met_x_da


#aoi_catd_da.plot.scatter(met_da)
#aoi_catd_da
c = b - a
c

In [ ]:
centroid = aoi_df.geometry[0].centroid
set_x, set_y, pixtype = (centroid.x, centroid.y, 'Center Pixel') 

ax = c.isel(band=0).sel(x=set_x, y=set_y, method='nearest').plot(figsize=(12,7))
plt.show()

